<a href="https://colab.research.google.com/github/heeyooon/TNT_2021_winter_team_3/blob/main/week5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import os

import warnings
import seaborn as sns
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler


#**1. 데이터 불러오기**

In [57]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sub = pd.read_csv('submission.csv')
train

,id,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5,count
0,3,20,16.3,1.0,1.5,89.0,576.0,0.027,76.0,33.0,49.0
1,6,13,20.1,0.0,1.4,48.0,916.0,0.042,73.0,40.0,159.0
2,7,6,13.9,0.0,0.7,79.0,1382.0,0.033,32.0,19.0,26.0
3,8,23,8.1,0.0,2.7,54.0,946.0,0.040,75.0,64.0,57.0
4,9,18,29.5,0.0,4.8,7.0,2000.0,0.057,27.0,11.0,431.0
...,...,...,...,...,...,...,...,...,...,...,...
1454,2174,4,16.8,0.0,1.6,53.0,2000.0,0.031,37.0,27.0,21.0
1455,2175,3,10.8,0.0,3.8,45.0,2000.0,0.039,34.0,19.0,20.0
1456,2176,5,18.3,0.0,1.9,54.0,2000.0,0.009,30.0,21.0,22.0
1457,2178,21,20.7,0.0,3.7,37.0,1395.0,0.082,71.0,36.0,216.0


In [58]:
x = train[['hour', 'hour_bef_temperature', 'hour_bef_precipitation',
       'hour_bef_windspeed', 'hour_bef_humidity', 'hour_bef_visibility',
       'hour_bef_ozone', 'hour_bef_pm10', 'hour_bef_pm2.5']]

y = train['count']

test = test[['hour', 'hour_bef_temperature', 'hour_bef_precipitation',
       'hour_bef_windspeed', 'hour_bef_humidity', 'hour_bef_visibility',
       'hour_bef_ozone', 'hour_bef_pm10', 'hour_bef_pm2.5']]


#**2. 데이터 전처리**

In [ ]:
x.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
## median imputation 
X = x.fillna(x.median())
test = test.fillna(x.median())

In [3]:
## correlation
sns.heatmap(x.corr())

NameError: ignored

In [59]:
## train test split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=1004)

In [60]:
## scaling

scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)
test = scaler.transform(test)

#**3. AutoML로 모델 고르기**

In [ ]:
# pip install pycaret
from pycaret.regression import *

In [ ]:
## make AutoML input

train_df = pd.DataFrame(data = X_train, columns=['hour', 'hour_bef_temperature', 'hour_bef_precipitation',
                                                 'hour_bef_windspeed', 'hour_bef_humidity', 'hour_bef_visibility',
                                                 'hour_bef_ozone', 'hour_bef_pm10', 'hour_bef_pm2.5'])

y_train = y_train.reset_index()
train_df['count'] = y_train['count']

test_df = pd.DataFrame(data = test, columns = ['hour', 'hour_bef_temperature', 'hour_bef_precipitation',
                                               'hour_bef_windspeed', 'hour_bef_humidity', 'hour_bef_visibility',
                                               'hour_bef_ozone', 'hour_bef_pm10', 'hour_bef_pm2.5'])

In [29]:
model = setup(data = train_df, target = 'count')

,Description,Value
0,session_id,2228
1,Target,count
2,Original Data,"(1167, 10)"
3,Missing Values,False
4,Numeric Features,9
5,Categorical Features,0
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(816, 9)"


In [30]:
compare_models(exclude=None,
               include = None,
               fold = 10,
               round=4,
               n_select = 1)


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,24.9229,1336.2480,36.2651,0.7943,0.5351,0.8104,0.931
et,Extra Trees Regressor,25.2534,1364.8429,36.6614,0.7893,0.5166,0.7634,0.276
lightgbm,Light Gradient Boosting Machine,26.1024,1478.5620,38.2420,0.7699,0.5753,0.9152,0.042
gbr,Gradient Boosting Regressor,27.0732,1563.6850,39.2707,0.7603,0.5490,0.8712,0.064
xgboost,Extreme Gradient Boosting,26.9075,1580.5392,39.5011,0.7548,0.5623,0.8841,10.417
rf,Random Forest Regressor,27.5134,1608.9452,39.9205,0.7515,0.5525,0.9332,0.349
knn,K Neighbors Regressor,35.1262,2246.6068,47.1447,0.6511,0.6875,1.0548,0.038
ada,AdaBoost Regressor,41.0314,2535.7716,50.2870,0.6003,0.8097,1.8181,0.063
br,Bayesian Ridge,39.2281,2713.5496,51.9225,0.5770,0.8011,1.3462,0.011
lasso,Lasso Regression,39.1676,2715.6975,51.9459,0.5769,0.8029,1.3618,0.012


In [43]:
et = create_model('et')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,28.0649,1685.4714,41.0545,0.7768,0.3268,0.2838
1,26.4288,1396.5547,37.3705,0.8090,0.4635,0.4804
2,27.4009,1811.7129,42.5642,0.7659,0.4744,0.4965
3,22.6162,1109.2194,33.3049,0.8086,0.5307,0.7277
4,21.4834,953.3456,30.8763,0.8654,0.4092,0.4342
5,30.6578,1691.8666,41.1323,0.7693,0.6135,1.0684
6,20.6012,920.4002,30.3381,0.8570,0.5838,1.2499
7,22.4936,1156.3703,34.0054,0.7672,0.6661,1.2378
8,23.6747,1130.9709,33.6299,0.7395,0.6137,1.0039
9,29.1121,1792.5166,42.3381,0.7344,0.4840,0.6511


In [44]:
tuned_et = tune_model(et)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,38.0226,2766.9798,52.6021,0.6335,0.4565,0.4668
1,38.7195,2512.6099,50.1259,0.6563,0.7286,0.9767
2,36.0530,2255.3296,47.4903,0.7086,0.6664,0.8654
3,33.3348,1703.1103,41.2688,0.7061,0.7487,1.1841
4,31.9198,1907.0596,43.6699,0.7307,0.6903,1.1631
5,37.3158,2497.2713,49.9727,0.6595,0.7281,1.4924
6,30.8761,1675.7375,40.9358,0.7397,0.8033,2.3342
7,33.1218,2045.8657,45.2312,0.5882,0.8399,1.7827
8,32.3046,1749.4437,41.8264,0.5970,0.8171,1.8949
9,37.4232,2742.6009,52.3698,0.5936,0.6561,1.3315


In [ ]:
et_final = finalize_model(tuned_et)
pred_df = predict_model(et_final,test_df)

In [38]:
pred_df

,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5,Label
0,-0.674189,0.768491,-0.183772,-0.871474,0.482381,-0.788404,0.101156,-0.421022,-0.235332,104.513799
1,0.773958,2.546181,-0.183772,2.149350,-0.945956,0.303967,1.150705,-0.253513,0.392035,218.361389
2,0.194699,0.443536,5.450589,-0.282045,2.107731,-2.095470,-1.000871,-0.689037,-0.165625,87.336941
3,-0.819003,1.112560,-0.183772,0.012670,0.383875,-0.391646,-0.633529,-0.153007,0.531450,77.874986
4,1.498031,-0.397521,5.450589,0.675778,2.009225,-2.052531,0.101156,-1.292072,-1.071822,72.985716
...,...,...,...,...,...,...,...,...,...,...
710,-1.543076,1.513974,-0.183772,-0.061009,0.383875,0.570189,0.004017,0.020156,-0.003407,88.710385
711,-1.543076,0.271502,-0.183772,-1.092510,0.137610,1.008168,0.004017,0.020156,-0.003407,62.108577
712,-0.384559,1.265479,-0.183772,-0.134688,0.679394,0.645762,-1.000871,-1.325574,-1.071822,116.353418
713,0.629143,1.972732,-0.183772,-0.650438,-0.305667,0.932595,-0.371142,-0.555030,-0.305040,138.182874


In [46]:
sub['count'] = pred_df['Label']
sub.to_csv('0128_submission13.csv', index = False)

## extraTreeRegressor submission score : 62.4111521154

#**4. CatBoostRegressor**

In [61]:
from catboost import CatBoostRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

from scipy.stats import randint
from random import uniform

def catboost_pipline(X_train_data, X_test_data, y_train_data, y_test_data, 
                     model, param_dist, cv=5, scoring_fit = 'neg_mean_absolute_error'):

    n_iter_search = 50
    rs = RandomizedSearchCV(estimator = model, 
                            param_distributions = param_dist,
                            n_iter = n_iter_search,
                            cv = 10,
                            n_jobs = 1,
                            scoring = scoring_fit,
                            random_state = 220,
                            verbose = 10,
                            # return_train_score = True
                           )

    fitted_model = rs.fit(X_train_data, y_train_data,
                          verbose = False,
                          eval_set = (X_test_data, y_test_data),
                          early_stopping_rounds = 200,
                         )
    

    print("BEST PARAMS : ", rs.best_params_)
    print("BEST SCORE : ", rs.best_score_)

    pred = fitted_model.predict(X_test_data)
    
    return fitted_model, pred, rs


param_dist = {'learning_rate' : np.linspace(0.05, 0.1), #uniform(0.05, 0.1)
              'depth' : randint(5, 100),
              'iterations' : randint(1000, 3000)
             }

model = CatBoostRegressor(random_seed = 220,
                          verbose = True,
                          use_best_model = True,
                          eval_metric = 'RMSE',
                          od_type = 'Iter',
                          od_wait = 100,
                         )
                          

model, pred, rs = catboost_pipline(X_train, X_val, 
                                   y_train, y_val, 
                                   model, param_dist)

Fitting 10 folds for each of 50 candidates, totalling 500 fits
[CV] depth=97, iterations=1015, learning_rate=0.05204081632653061 ....
[CV]  depth=97, iterations=1015, learning_rate=0.05204081632653061, score=nan, total=   0.0s
[CV] depth=97, iterations=1015, learning_rate=0.05204081632653061 ....
[CV]  depth=97, iterations=1015, learning_rate=0.05204081632653061, score=nan, total=   0.0s
[CV] depth=97, iterations=1015, learning_rate=0.05204081632653061 ....
[CV]  depth=97, iterations=1015, learning_rate=0.05204081632653061, score=nan, total=   0.0s
[CV] depth=97, iterations=1015, learning_rate=0.05204081632653061 ....
[CV]  depth=97, iterations=1015, learning_rate=0.05204081632653061, score=nan, total=   0.0s
[CV] depth=97, iterations=1015, learning_rate=0.05204081632653061 ....
[CV]  depth=97, iterations=1015, learning_rate=0.05204081632653061, score=nan, total=   0.0s
[CV] depth=97, iterations=1015, learning_rate=0.05204081632653061 ....
[CV]  depth=97, iterations=1015, learning_rate

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.0s remaining:    0.0s


[CV]  depth=14, iterations=2647, learning_rate=0.05, score=-20.854, total= 4.6min
[CV] depth=14, iterations=2647, learning_rate=0.05 ...................
[CV]  depth=14, iterations=2647, learning_rate=0.05, score=-25.083, total= 4.7min
[CV] depth=14, iterations=2647, learning_rate=0.05 ...................
[CV]  depth=14, iterations=2647, learning_rate=0.05, score=-24.350, total= 4.6min
[CV] depth=14, iterations=2647, learning_rate=0.05 ...................
[CV]  depth=14, iterations=2647, learning_rate=0.05, score=-30.193, total= 4.6min
[CV] depth=14, iterations=2647, learning_rate=0.05 ...................
[CV]  depth=14, iterations=2647, learning_rate=0.05, score=-29.940, total= 4.6min
[CV] depth=14, iterations=2647, learning_rate=0.05 ...................
[CV]  depth=14, iterations=2647, learning_rate=0.05, score=-27.793, total= 3.6min
[CV] depth=14, iterations=2647, learning_rate=0.05 ...................
[CV]  depth=14, iterations=2647, learning_rate=0.05, score=-24.801, total= 4.6min


[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed: 100.8min finished


BEST PARAMS :  {'depth': 7, 'iterations': 2669, 'learning_rate': 0.056122448979591844}
BEST SCORE :  -24.88336760754829


In [ ]:
sub['count'] = model.predict(test)
sub.to_csv('0128_submission14.csv', index = False)

## catBoostRegressor submission score : 49.9389487973

#**5. LigthGBMRegressor**

In [64]:
# LightGBM parameters
import lightgbm as lgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import StratifiedKFold


def lgbm_pipline(X_train_data, X_test_data, y_train_data, y_test_data, 
                 model, param_dist, cv=5, scoring_fit='neg_mean_absolute_error'):
   
    n_iter_search = 200
    rs = RandomizedSearchCV(estimator = model, 
                            param_distributions = param_dist,
                            n_iter = n_iter_search,
                            cv = 5,
                            n_jobs = -1,
                            scoring = scoring_fit,
                            random_state = 220,
                            verbose = 10,
                           )
    
    fitted_model = rs.fit(X_train_data, y_train_data, 
                          verbose = False, early_stopping_rounds = 200,
                          eval_metric = 'mae', eval_set = [(X_test_data, y_test_data)]
                         )
    
    print("BEST SCORE : ", rs.best_score_)
    print("BEST PARAMS : ", rs.best_params_)
    
    pred = fitted_model.predict(X_test_data)
    print("VAL MEA SCORE : ", mean_absolute_error(y_test_data, pred))
    
    return fitted_model, pred, rs


from scipy.stats import randint

param_dist = {'max_depth' : randint(0, 100), # 과적합 시 줄이기, 가장 먼저 설정
              'num_leaves' : randint(10, 100), # smaller than 2^(max_depth)
              'min_data_in_leaf' : randint(10, 200), # prevent overfitting
              'num_iteration' : randint(1000, 3000),
              'learning_rate' : np.linspace(0.05, 0.1),
              'feature_fraction' : np.linspace(0.5, 0.9),
              'bagging_fraction' : np.linspace(0.7, 0.9),
             }

model = lgb.LGBMRegressor(task ='train',
                          boosting_type='gbdt',  
                          objective='regression',
                          metric='mae',
                          random_state=220) 


model, pred, rs = lgbm_pipline(X_train, X_val, 
                               y_train, y_val, 
                               model, param_dist)

Fitting 5 folds for each of 200 candidates, totalling 1000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1495s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Done  58 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done  76 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done  98 tasks      | elapsed:    6.5s
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:    7.5s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:    8.8s
[Parallel(n_jobs=-1)]: Done 172 tasks      | elapsed:   10.5s
[Parallel(n_jobs=-1)]: Done 202 tasks      | elapsed:   12.2s
[Parallel(n_jobs=-1)]: Done 232 tasks      | elapsed:   13.9s
[Parallel(n_jobs=-1)]: Done 266 tasks      | elapsed:   16.3s
[Parallel(n_jobs=-1)]: Done 300 tas

BEST SCORE :  -26.795810593634105
BEST PARAMS :  {'bagging_fraction': 0.8591836734693877, 'feature_fraction': 0.8020408163265307, 'learning_rate': 0.0816326530612245, 'max_depth': 0, 'min_data_in_leaf': 22, 'num_iteration': 1115, 'num_leaves': 25}
VAL MEA SCORE :  26.283978946084698


[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:  1.1min finished


In [ ]:
sub['count'] = model.predict(test)
sub.to_csv('0126_submission16.csv', index = False)

## catBoostRegressor submission score : 46.2884239187

**++) stacking**

In [66]:
## data for stacking
y_train_v = y_train.values

In [46]:
from mlxtend.regressor import StackingCVRegressor

from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import  GradientBoostingRegressor
from sklearn.linear_model import LinearRegression

from sklearn.model_selection import cross_val_score
import numpy as np



RANDOM_SEED = 220

# models 
xgb = xgb.XGBRegressor(booster = 'gbtree', eval_metric = "mae", random_state = RANDOM_SEED, verbosity = 0)
lgb = lgb.LGBMRegressor(task = 'train', boosting_type='gbdt',  objective='regression', metric='mae', random_state = RANDOM_SEED)

# meta model              
# lr = LinearRegression()
rf = RandomForestRegressor(random_state = RANDOM_SEED, n_estimators = 500)


# Starting from v0.16.0, StackingCVRegressor supports
# `random_state` to get deterministic result.
stack = StackingCVRegressor(regressors = (xgb, lgb),
                            meta_regressor = rf,
                            # random_state=RANDOM_SEED
                            )

print('5-fold cross validation scores:\n')

for clf, label in zip([xgb, lgb, stack], ['XGBoost', 'LightGBM', 'StackingCVRegressor']):
    scores = cross_val_score(clf, X_train, y_train_v, cv = 5)
    print("R^2 Score: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

5-fold cross validation scores:

R^2 Score: 0.76 (+/- 0.02) [XGBoost]
R^2 Score: 0.77 (+/- 0.02) [LightGBM]
R^2 Score: 0.74 (+/- 0.02) [StackingCVRegressor]


In [47]:
stack.fit(X_train, y_train_v)
y_val_pred = stack.predict(X_val)
mean_absolute_error(y_val, y_val_pred)

29.610787671232877

In [ ]:
sub['count'] = stack.predict(test)
sub.to_csv('0128_submission7.csv', index = False)

# stacking submission score : 51.5444179231